In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import pickle 
import csv
import math

In [17]:
#Loading ISTAT DATA
istatdata = pd.read_csv("Desktop\DCCV_AVQ_PERSONE_07062022160135090.csv")

In [18]:
istatdata

,ITTER107,Territorio,TIPO_DATO_AVQ,Tipo dato,MISURA_AVQ,Misura,TIME,Seleziona periodo,Value,Flag Codes,Flags
0,IT,Italia,3_SPORT_OCCAS,in modo saltuario,THV,valori in migliaia,2020,2020,5559.0,NaN,NaN
1,IT,Italia,3_SPORT_OCCAS,in modo saltuario,THV,valori in migliaia,2021,2021,6368.0,NaN,NaN
2,IT,Italia,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2020,2020,27.1,NaN,NaN
3,IT,Italia,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2021,2021,23.6,NaN,NaN
4,IT,Italia,3_PHYS_ACTIV,solo qualche attività fisica,THV,valori in migliaia,2020,2020,16419.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
571,ITD2,Provincia Autonoma Trento,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,12.5,NaN,NaN
572,ITE1,Toscana,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2020,2020,29.6,NaN,NaN
573,ITE1,Toscana,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,26.8,NaN,NaN
574,ITE,Centro,3_SPORT_OCCAS,in modo saltuario,HSC,per 100 persone con le stesse caratteristiche,2020,2020,9.8,NaN,NaN


In [19]:
#Working with ISTAT DATA from 2021
istatdata21 = istatdata[istatdata.TIME==2021]
istatdata21 = istatdata21[istatdata21.MISURA_AVQ=="HSC"]

In [20]:
#Normalizing physical activity level per region within a 0-1 range
istatdata21 = istatdata[istatdata.TIME==2021]
istatdata21 = istatdata21[istatdata21.MISURA_AVQ=="HSC"]
istatdata21 = istatdata21.copy(deep=True)
multiplier = []
ranktomultiplier = {}


ranktomultiplier['non praticano sport, né attività fisica'] = 0
ranktomultiplier["solo qualche attività fisica"] = 1
ranktomultiplier["in modo saltuario"] = 2
ranktomultiplier["in modo continuativo"] = 3

istatdata21 = istatdata21.copy(deep=True)

for el in istatdata21["Tipo dato"]:
    multiplier.append(ranktomultiplier[el]) 
istatdata21["multiplier"] = multiplier
istatdata21["score"] = istatdata21["Value"] * istatdata21["multiplier"] 

#GROUP BY TERRITORIES AND SUMMING THE SCORE WITH THE DECIDED METRIC
level = istatdata21.groupby('Territorio')['score'].sum()

#CONVERTING TO DATA FRAME
physical_data = level.to_frame()
#MIN-MAX NORMALIZATION ()
maxvalue = 300
minvalue = 0
physical_data["score"] = round((physical_data["score"] - minvalue) / (maxvalue - minvalue),3)

In [21]:
istatdata21

,ITTER107,Territorio,TIPO_DATO_AVQ,Tipo dato,MISURA_AVQ,Misura,TIME,Seleziona periodo,Value,Flag Codes,Flags,multiplier,score
3,IT,Italia,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2021,2021,23.6,NaN,NaN,3,70.8
7,IT,Italia,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,33.7,NaN,NaN,0,0.0
11,IT,Italia,3_PHYS_ACTIV,solo qualche attività fisica,HSC,per 100 persone con le stesse caratteristiche,2021,2021,31.7,NaN,NaN,1,31.7
15,IT,Italia,3_SPORT_OCCAS,in modo saltuario,HSC,per 100 persone con le stesse caratteristiche,2021,2021,10.9,NaN,NaN,2,21.8
17,ITF2,Molise,3_SPORT_OCCAS,in modo saltuario,HSC,per 100 persone con le stesse caratteristiche,2021,2021,8.2,NaN,NaN,2,16.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,ITDA,Trentino Alto Adige / Südtirol,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,13.5,NaN,NaN,0,0.0
569,ITF3,Campania,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2021,2021,14.3,NaN,NaN,3,42.9
571,ITD2,Provincia Autonoma Trento,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,12.5,NaN,NaN,0,0.0
573,ITE1,Toscana,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,26.8,NaN,NaN,0,0.0


In [22]:
nordlist = ["lombardia","emilia","trento","bolzano","friuli","liguria","aosta","piemonte","veneto"]
Northen_Italy = []
for x in physical_data.index:
    for y in nordlist:
        if y in x.lower():
            Northen_Italy.append(x)

In [23]:
Northen_Italy

['Emilia-Romagna',
 'Friuli-Venezia Giulia',
 'Liguria',
 'Lombardia',
 'Piemonte',
 'Provincia Autonoma Bolzano / Bozen',
 'Provincia Autonoma Trento',
 "Valle d'Aosta / Vallée d'Aoste",
 'Veneto']

In [24]:
#ORDER BY SCORE DESC
physical_data = physical_data.sort_values(by="score",ascending=False)

In [25]:
physical_data["Territorio"] = physical_data.index

In [26]:
physical_data[physical_data["Territorio"].isin(Northen_Italy)]

,score,Territorio
Territorio,,
Provincia Autonoma Bolzano / Bozen,0.642,Provincia Autonoma Bolzano / Bozen
Provincia Autonoma Trento,0.561,Provincia Autonoma Trento
Valle d'Aosta / Vallée d'Aoste,0.531,Valle d'Aosta / Vallée d'Aoste
Veneto,0.489,Veneto
Lombardia,0.488,Lombardia
Emilia-Romagna,0.467,Emilia-Romagna
Friuli-Venezia Giulia,0.456,Friuli-Venezia Giulia
Liguria,0.447,Liguria
Piemonte,0.446,Piemonte


In [27]:
data2021 = physical_data.copy(deep=True)

In [28]:
istatdata21

,ITTER107,Territorio,TIPO_DATO_AVQ,Tipo dato,MISURA_AVQ,Misura,TIME,Seleziona periodo,Value,Flag Codes,Flags,multiplier,score
3,IT,Italia,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2021,2021,23.6,NaN,NaN,3,70.8
7,IT,Italia,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,33.7,NaN,NaN,0,0.0
11,IT,Italia,3_PHYS_ACTIV,solo qualche attività fisica,HSC,per 100 persone con le stesse caratteristiche,2021,2021,31.7,NaN,NaN,1,31.7
15,IT,Italia,3_SPORT_OCCAS,in modo saltuario,HSC,per 100 persone con le stesse caratteristiche,2021,2021,10.9,NaN,NaN,2,21.8
17,ITF2,Molise,3_SPORT_OCCAS,in modo saltuario,HSC,per 100 persone con le stesse caratteristiche,2021,2021,8.2,NaN,NaN,2,16.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,ITDA,Trentino Alto Adige / Südtirol,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,13.5,NaN,NaN,0,0.0
569,ITF3,Campania,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2021,2021,14.3,NaN,NaN,3,42.9
571,ITD2,Provincia Autonoma Trento,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,12.5,NaN,NaN,0,0.0
573,ITE1,Toscana,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,26.8,NaN,NaN,0,0.0


In [29]:
istatdata211 = istatdata[istatdata.TIME==2021]
istatdata211

,ITTER107,Territorio,TIPO_DATO_AVQ,Tipo dato,MISURA_AVQ,Misura,TIME,Seleziona periodo,Value,Flag Codes,Flags
1,IT,Italia,3_SPORT_OCCAS,in modo saltuario,THV,valori in migliaia,2021,2021,6368.0,NaN,NaN
3,IT,Italia,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2021,2021,23.6,NaN,NaN
5,IT,Italia,3_PHYS_ACTIV,solo qualche attività fisica,THV,valori in migliaia,2021,2021,18516.0,NaN,NaN
7,IT,Italia,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,33.7,NaN,NaN
9,IT,Italia,3_SPORT_CONT,in modo continuativo,THV,valori in migliaia,2021,2021,13769.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
567,ITDA,Trentino Alto Adige / Südtirol,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,13.5,NaN,NaN
569,ITF3,Campania,3_SPORT_CONT,in modo continuativo,HSC,per 100 persone con le stesse caratteristiche,2021,2021,14.3,NaN,NaN
571,ITD2,Provincia Autonoma Trento,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,12.5,NaN,NaN
573,ITE1,Toscana,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2021,2021,26.8,NaN,NaN


In [30]:
#DOING THE SAME FOR 2020
istatdata21 = istatdata[istatdata.TIME==2020]
istatdata21 = istatdata21[istatdata21.MISURA_AVQ=="HSC"]
istatdata21 = istatdata21.copy(deep=True)
multiplier = []
ranktomultiplier = {}


ranktomultiplier['non praticano sport, né attività fisica'] = 0
ranktomultiplier["solo qualche attività fisica"] = 1
ranktomultiplier["in modo saltuario"] = 2
ranktomultiplier["in modo continuativo"] = 3

for el in istatdata21["Tipo dato"]:
    multiplier.append(ranktomultiplier[el]) 
istatdata21["multiplier"] = multiplier
istatdata21["score"] = istatdata21["Value"] * istatdata21["multiplier"] 

#GROUP BY TERRITORIES AND SUMMING THE SCORE WITH THE DECIDED METRIC
level = istatdata21.groupby('Territorio')['score'].sum()

#CONVERTING TO DATA FRAME
physical_data = level.to_frame()
#MIN-MAX NORMALIZATION ()
maxvalue = 300
minvalue = 0
physical_data["score"] = round((physical_data["score"] - minvalue) / (maxvalue - minvalue),3)

#ORDER BY SCORE DESC
physical_data = physical_data.sort_values(by="score",ascending=False)

physical_data["Territorio"] = physical_data.index


data2020 = physical_data.copy(deep=True)



In [31]:
data2020[data2020["Territorio"].isin(Northen_Italy)]

,score,Territorio
Territorio,,
Provincia Autonoma Bolzano / Bozen,0.636,Provincia Autonoma Bolzano / Bozen
Provincia Autonoma Trento,0.561,Provincia Autonoma Trento
Valle d'Aosta / Vallée d'Aoste,0.529,Valle d'Aosta / Vallée d'Aoste
Veneto,0.512,Veneto
Lombardia,0.505,Lombardia
Friuli-Venezia Giulia,0.499,Friuli-Venezia Giulia
Emilia-Romagna,0.487,Emilia-Romagna
Liguria,0.470,Liguria
Piemonte,0.448,Piemonte


In [32]:
istatdata[istatdata["Territorio"].isin(Northen_Italy)]

,ITTER107,Territorio,TIPO_DATO_AVQ,Tipo dato,MISURA_AVQ,Misura,TIME,Seleziona periodo,Value,Flag Codes,Flags
34,ITC3,Liguria,3_PHYS_ACTIV,solo qualche attività fisica,HSC,per 100 persone con le stesse caratteristiche,2020,2020,35.4,NaN,NaN
35,ITC3,Liguria,3_PHYS_ACTIV,solo qualche attività fisica,HSC,per 100 persone con le stesse caratteristiche,2021,2021,38.6,NaN,NaN
44,ITD2,Provincia Autonoma Trento,3_SPORT_CONT,in modo continuativo,THV,valori in migliaia,2020,2020,178.0,NaN,NaN
45,ITD2,Provincia Autonoma Trento,3_SPORT_CONT,in modo continuativo,THV,valori in migliaia,2021,2021,171.0,NaN,NaN
52,ITC3,Liguria,3_SPORT_OCCAS,in modo saltuario,THV,valori in migliaia,2020,2020,158.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
559,ITD3,Veneto,3_PHYS_ACTIV,solo qualche attività fisica,THV,valori in migliaia,2021,2021,1633.0,NaN,NaN
564,ITD3,Veneto,3_SPORT_OCCAS,in modo saltuario,HSC,per 100 persone con le stesse caratteristiche,2020,2020,12.3,NaN,NaN
565,ITD3,Veneto,3_SPORT_OCCAS,in modo saltuario,HSC,per 100 persone con le stesse caratteristiche,2021,2021,14.4,NaN,NaN
570,ITD2,Provincia Autonoma Trento,3_NEV_SPORT,"non praticano sport, né attività fisica",HSC,per 100 persone con le stesse caratteristiche,2020,2020,17.3,NaN,NaN


In [33]:
data202021

NameError: name 'data202021' is not defined

In [34]:
#Generating a standar deviation of physical activity for simulation purposes
#Sd based on the mean between 2020 and 2021 data
#Computing mean
avg = (data2020.score + data2021.score)/2

#Computing standard deviation
#Deviation from mean for the two years
devdata2020 = data2020.score - avg
devdata2021 = data2021.score - avg
#standard deviation
devstdata = np.sqrt((devdata2020**2+devdata2021**2)/2)

In [35]:
#Generating average data by initially copying the 2021 dataset
data20202021 = data2021.copy()
#Replacing the 2021 score with the mean score
data20202021.score = avg
#Adding standard deviation 
data20202021['dev_st'] = devstdata

In [36]:
#Dropping index and visualizing data
data20202021.reset_index(drop=True,inplace=True)
data20202021

,score,Territorio,dev_st
0,0.6390,Provincia Autonoma Bolzano / Bozen,0.0030
1,0.5990,Trentino Alto Adige / Südtirol,0.0020
2,0.5610,Provincia Autonoma Trento,0.0000
3,0.5300,Valle d'Aosta / Vallée d'Aoste,0.0010
4,0.5005,Veneto,0.0115
5,0.4965,Lombardia,0.0085
6,0.4980,Nord-est,0.0110
7,0.4875,Nord,0.0085
8,0.4795,Nord-ovest,0.0065
9,0.4770,Emilia-Romagna,0.0100


In [37]:
#Rounding the standard deviation up to allow more variability on the data
data20202021.dev_st = (np.ceil(data20202021.dev_st*1000))/1000

#To avoid to have regions without variability, I will impute 
#standard deviation equal to the mean standard deviation to all the rows with sd = 0

meanstdbyscore = data20202021.dev_st / data20202021.score
#Computing mean sd ignoring cases where sd=0
meanstdbyscore = [x for x in meanstdbyscore if x!=0]
#mean sd
meanstdweight = np.ceil(np.mean(meanstdbyscore)*100)/100

#Creating new standar deviation column with imputed sd where sd was 0
deviazioni = list(data20202021.dev_st)
deviazioni2 =  list(data20202021.dev_st)
gliscore = list(data20202021.score)
for i in range(len(deviazioni)):
    if deviazioni[i]==0:
        deviazioni2[i] = meanstdweight * gliscore[i]
#new column
data20202021.dev_st = np.round(deviazioni2,3)
#visualizing
data20202021.head()

,score,Territorio,dev_st
0,0.6390,Provincia Autonoma Bolzano / Bozen,0.004
1,0.5990,Trentino Alto Adige / Südtirol,0.003
2,0.5610,Provincia Autonoma Trento,0.017
3,0.5300,Valle d'Aosta / Vallée d'Aoste,0.002
4,0.5005,Veneto,0.012


# Daily steps table 

The previously created index takes 0 when there is no physical activity, 0.5 when there is "average" physical activity and 1 when there is optimal physical activity. Now, we will use this index to model the physical activity standards in different fields (cardio, squat, pushups etc.)

According to scientific studies from the Standford University (https://cs.stanford.edu/people/jure/pubs/activity-inequality-nature17.pdf), the average daily steps in Italy are 5296. 
So, this number will used as the "average" physical activity standard (physical activity score = 0.5). Hence, a physical activity score of 1 will correspond to 5296 * 2 daily steps and 0 will corespond to 5296 * 0 daily steps. So, each value will be multiplied by (5296 * 2) to generate the "proportion" of "optimal physical activity done" for each region.

In [38]:
#Average daily steps
avgsteps = 5296

In [39]:
#Generating daily steps by region
dailysteps = np.around(data20202021.score*(avgsteps*2))
dailystepssd = np.around(data20202021.dev_st*(avgsteps*2))

In [66]:
#Generating daily steps by region dataset
dailysteps_df = pd.DataFrame({'Region':data20202021.Territorio,'Daily_steps':dailysteps,'steps_sd':dailystepssd})

In [41]:
dailysteps_df.head()

,Region,Daily_steps,sd
0,Provincia Autonoma Bolzano / Bozen,6768.0,42.0
1,Trentino Alto Adige / Südtirol,6345.0,32.0
2,Provincia Autonoma Trento,5942.0,180.0
3,Valle d'Aosta / Vallée d'Aoste,5614.0,21.0
4,Veneto,5301.0,127.0


In [42]:
dailysteps_df.tail()

,Region,Daily_steps,sd
31,Sud,3347.0,85.0
32,Basilicata,3231.0,42.0
33,Calabria,3167.0,42.0
34,Sicilia,3008.0,53.0
35,Campania,2934.0,42.0


# Daily push-ups table

According to the Muscular Fitness Test (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7531937/), average push-ups values for men and women are, respectively between 28 and 32 and between 8 and 14 daily pushups. Considering that women population is around 51.3 % of the total population ( https://data.worldbank.org/indicator/SP.POP.TOTL.FE.ZS?end=2021&locations=IT&start=1960&view=chart ), we can compute the mean daily pushups for the general population through weighted mean.

In [43]:
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7531937/
avgpushupmen = (28+32)/2
avgpushupwomen = (8+14)/2
fweight = 0.513 #female population weight
mweight = 1 - fweight #male population weight
avgpushupgeneral = math.ceil((avgpushupmen*mweight)+(avgpushupwomen*fweight))

In [44]:
avgpushupgeneral

21

In [45]:
dailypushups = np.around(data20202021.score*(avgpushupgeneral*2))
dailypushupssd = np.around(data20202021.dev_st*(avgpushupgeneral*2))
dailypushupssd+=1 #to add variation since there are null variances 

In [63]:
dailypushups_df = pd.DataFrame({'Region':data20202021.Territorio,'Daily_pushups':dailypushups,'pushups_sd':dailypushupssd})

In [64]:
dailypushups_df

,Region,Daily_pushups,pushups_sd
0,Provincia Autonoma Bolzano / Bozen,27.0,1.0
1,Trentino Alto Adige / Südtirol,25.0,1.0
2,Provincia Autonoma Trento,24.0,2.0
3,Valle d'Aosta / Vallée d'Aoste,22.0,1.0
4,Veneto,21.0,2.0
5,Lombardia,21.0,1.0
6,Nord-est,21.0,2.0
7,Nord,20.0,1.0
8,Nord-ovest,20.0,1.0
9,Emilia-Romagna,20.0,1.0


# Daily  Body squats table (Lower body strength physical activity)

According to the Muscular Fitness Test (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7531937/), average squat values for men and women are, respectively between 40-42 and between 8 and 33-38 daily squats. Considering that women population is around 51.3 % of the total population ( https://data.worldbank.org/indicator/SP.POP.TOTL.FE.ZS?end=2021&locations=IT&start=1960&view=chart ), we can compute the mean daily pushups for the general population through weighted mean.

In [48]:

avgsquatsmen = (40+42)/2
avgsquatswomen = (33+38)/2
fweight = 0.513 #female population weight
mweight = 1 - fweight #male population weight
avgsquatsgeneral = math.ceil((avgsquatsmen*mweight)+(avgsquatswomen*fweight))

In [49]:
dailysquats = np.around(data20202021.score*(avgsquatsgeneral*2))
dailysquatssd = np.around(data20202021.dev_st*(avgsquatsgeneral*2))
dailysquatssd+=1 #to add variation since there are null variances 

In [50]:
dailysquatssd

0     1.0
1     1.0
2     2.0
3     1.0
4     2.0
5     2.0
6     2.0
7     2.0
8     2.0
9     2.0
10    1.0
11    3.0
12    2.0
13    1.0
14    2.0
15    1.0
16    2.0
17    2.0
18    1.0
19    1.0
20    1.0
21    2.0
22    2.0
23    2.0
24    2.0
25    1.0
26    2.0
27    2.0
28    2.0
29    1.0
30    3.0
31    2.0
32    1.0
33    1.0
34    1.0
35    1.0
Name: dev_st, dtype: float64

In [61]:
dailysquats_df = pd.DataFrame({'Region':data20202021.Territorio,'Daily_squats':dailysquats,'squats_sd':dailysquatssd})

In [62]:
dailysquats_df

,Region,Daily_squats,squats_sd
0,Provincia Autonoma Bolzano / Bozen,50.0,1.0
1,Trentino Alto Adige / Südtirol,47.0,1.0
2,Provincia Autonoma Trento,44.0,2.0
3,Valle d'Aosta / Vallée d'Aoste,41.0,1.0
4,Veneto,39.0,2.0
5,Lombardia,39.0,2.0
6,Nord-est,39.0,2.0
7,Nord,38.0,2.0
8,Nord-ovest,37.0,2.0
9,Emilia-Romagna,37.0,2.0


In [76]:
#AGGREGATING ALL THE PHYSICAL ACTIVITY DATA INTO ONE SINGLE TABLE
activitydf = pd.concat([dailysquats_df,dailypushups_df,dailysteps_df],axis=1)
activitydf = activitydf.loc[:,~activitydf.columns.duplicated()].copy()
activitydf

,Region,Daily_squats,squats_sd,Daily_pushups,pushups_sd,Daily_steps,steps_sd
0,Provincia Autonoma Bolzano / Bozen,50.0,1.0,27.0,1.0,6768.0,42.0
1,Trentino Alto Adige / Südtirol,47.0,1.0,25.0,1.0,6345.0,32.0
2,Provincia Autonoma Trento,44.0,2.0,24.0,2.0,5942.0,180.0
3,Valle d'Aosta / Vallée d'Aoste,41.0,1.0,22.0,1.0,5614.0,21.0
4,Veneto,39.0,2.0,21.0,2.0,5301.0,127.0
5,Lombardia,39.0,2.0,21.0,1.0,5259.0,95.0
6,Nord-est,39.0,2.0,21.0,2.0,5275.0,127.0
7,Nord,38.0,2.0,20.0,1.0,5164.0,95.0
8,Nord-ovest,37.0,2.0,20.0,1.0,5079.0,74.0
9,Emilia-Romagna,37.0,2.0,20.0,1.0,5052.0,106.0


In [107]:
#CREATING A LOCIAL LIST "RETREG" WHICH TAKES TRUE IF A ROW IS A REGION AND FALSE OTHERWISE
retreg = []
for x in activitydf.Region:
    textlow = x.lower()
    stopwords_list = ['trento','bolzano','0','area','mezzogiorno','nord','sud','mezzogiorno','isole','centro','italia']
    conditionmet = True
    for el in stopwords_list:
        if el in textlow:
            conditionmet = False
    if conditionmet:
        retreg.append(True)
    else:
        retreg.append(False)

In [109]:
#Adding RETREG to the created dataset
activitydf['isregion'] = retreg

In [114]:
#Delete non-region data
activitydf = activitydf[activitydf['isregion']==True].copy()

In [117]:
#Regional data
activitydf

,Region,Daily_squats,squats_sd,Daily_pushups,pushups_sd,Daily_steps,steps_sd,isregion
1,Trentino Alto Adige / Südtirol,47.0,1.0,25.0,1.0,6345.0,32.0,True
3,Valle d'Aosta / Vallée d'Aoste,41.0,1.0,22.0,1.0,5614.0,21.0,True
4,Veneto,39.0,2.0,21.0,2.0,5301.0,127.0,True
5,Lombardia,39.0,2.0,21.0,1.0,5259.0,95.0,True
9,Emilia-Romagna,37.0,2.0,20.0,1.0,5052.0,106.0,True
10,Toscana,36.0,1.0,19.0,1.0,4904.0,21.0,True
11,Friuli-Venezia Giulia,37.0,3.0,20.0,2.0,5058.0,233.0,True
12,Liguria,36.0,2.0,19.0,2.0,4856.0,127.0,True
13,Piemonte,35.0,1.0,19.0,1.0,4735.0,21.0,True
15,Marche,34.0,1.0,18.0,1.0,4660.0,42.0,True


DATA MODEL

TABLE = REGIONS

ATTRIBUTES:

TERRITORIO ( <STRING> )

SCORE (<FLOAT>) (RANGE 0-1)
    
DEV_ST (<FLOAT>)
